# k-Nearest Neighbors Classification
## DS-3001: Machine Learning 1

Content adapted from Terence Johnson (UVA)

**Notebook Summary**: In this notebook we start to look at a basic machine learning model: k Nearest Neighbors. We talk about supervised vs unsupervised learning, classification vs regression, the KNN Classification Model, data prepration, hyperparameter tuning, overfitting vs underfitting, and model evaluation.

#### Setting Up Google Colab Environment

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os # For changing directory

# Load in from Sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

# To mount your google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# path_to_DS_3001_folder = '/content/drive/MyDrive/DS-3001/02_Intro_to_ML_Algorithms'
path_to_DS_3001_folder = ''

# Update the path to your folder for the class
# Where you stored the data from the previous noteboook
# path_to_DS_3001_folder = ''
os.chdir(path_to_DS_3001_folder)

## Supervised Learning

* *Supervised learning* is a type of machine learning where **we have the true values for what the outcome should be for each observation in our training data set**. Ex. If training a model to predict whether a patient has diabetes or not, our training data has knowledge of whether the patient has diabetes or not.

* *Unsupervised learning* is type of machine learning where **we do not have access to the true labels of our data even in our training data set**. Ex. We have data for cells and we want to identify different cell types, but we don't know what those cell types are beforehand. This is something we'll discuss in the coming weeks.

* The general set up for supervised learning is:
  - There is a target/outcome variable $y$, which we are trying to predict using features/covaraites $X$.
  - Our machine learning models have the following premise: "If $X$ and $X'$ are similar, then $y$ and $y'$ are probably similar."
  - A **predictive model** is the mapping from features/covariates $\hat{x}$ to target/outcome $\hat{y}$, so $\hat{y} = m(\hat{x})$ where $m()$ is our model.

## Classification vs Regression

* Whether the task is **classification** or **regression** depends on the data type of your output prediction.

* **Classification:** When we are predicting a categorical variable. Ex. predicting whether a patient has diabetes or not.

* **Regression:** When we are predicting a numeric variable. Usually conitnuous. Ex. Predicting the stock price of a company at a time point.

* We're going to use a simplistic model to explore both classification and regression. The model we're exploring is called **k Nearest Neighbors (k-NN or KNN)**.


## $k$-NN Classification

- Imagine we have a categorical variable $y$ of interest, that we want to predict based on features/covariates $X$: Disease based on patient symptoms, corporate or sovereign rating based on financial indicators, genus or species based on characteristics or DNA
- This is a classical **classification** problem: Given variables $X$, what is the probability we would assign to each possible label $\ell$ for the variable $y$?

- Imagine we have covariates/features $X = [x_1, x_2, ..., x_L]$, consisting of $N$ observations of $L$ variables, and observed outcomes/target variable $Y$
- Consider a new case $\hat{x} = (\hat{x}_1,...,\hat{x}_L)$. We want to make a guess of what **class/label** it will likely take, $\hat{\ell}$
- The *$k$ Nearest Neighbor Classification Algorithm* is:
  1. Compute the distance from $\hat{x}$ to each observation $x_i$ in the dataset
  2. Find the $k$ "nearest neighbors" $x_1^*$, $x_2^*$, ..., $x_k^*$ to $\hat{x}$ in the data in terms of distance, with outcome labels $\ell_{1}^*$, $\ell_2^*$, ..., $\ell_k^*$
  3. Return either a
        - **Hard Classification**: The modal/most common label among the nearest neighbor labels
        - **Soft Classification**: The proportions for which each of the labels occur among the nearest neighbors

### How do we define distance between our different observations?

- In one dimension, the distance between two values is the absolute value: $d(a,b) = |a-b| = \sqrt{(a-b)^2}$.

- In many dimensions, this is called the **Euclidean distance**. Imagine we have lists of $L$ variables for two observations, $a = [a_1, ..., a_L]$ and $b = [b_1, ..., b_L]$. We can then calculate the distance between a b as:

\begin{gather}
  d(a,b) = ||a - b|| \\ = \sqrt{(a_1 - b_1)^2 + (a_2-b_2)^2 + ... + (a_L - b_L)^2}
\end{gather}

In [ ]:
# Examples in code

# 1. 1-Dimension Example
# Distance between 8 and 12
print('Distance between 8 and 12:', np.abs(8-12))

# 2. 2-Dimension Example
# Distance bewteen (1,3) and (4,-2)
print('Distance between (1,3) and (4,-2):', np.sqrt((1-4)**2 + (3 - -2)**2))

# 3. Easier Way to Calculate Distances in 2D using Numpy
a = np.array([1, 3])
b = np.array([4, -2])

print('Distance between (1,3) and (4,-2), Numpy:', np.linalg.norm(a - b))

# 4. In 5-dimensions
a = np.array([10, 2, 0, 3, 24])
b = np.array([8, 10, 4, 9, 15])

print('Distance between 5-Dimensional Arrays:', np.linalg.norm(a-b))

### How do we consider the difference in scale for each of our variables?

- Often times, our variables will have different units and scales. For example, the variables we use may be age, height, weight which have units of years, feet, and lbs. These different units have different scales, which thus would affect our calcultions of distance unequally.

- Because of this, we need to normalize or scale each of our features so that they're on a similar scale and don't unequally affect the distance calculation.

- One way to do this is the **Min-Max scale**. For a variable $x$ we can compute the scaled version of the variable ($u$). For each observation i, we calculate:

\begin{gather}
  u_i = \frac{x_i - \text{min}(x)}{\text{max}(x) - \text{min}(x)}
\end{gather}

- This transforms all the values of $x$ so that that they lie between 0 and 1, with the smallest value mapped to 0 and the largest value mapped to 1.

- If you do not do this (or if the variables aren't already similarly scaled), neighbor methods simple won't work correctly. One variable may be unequally weighted over another.

- It's easier to transform the data and keep the original, untransformed version to compare with predictions later.


In [ ]:
# Let's create a MinMaxScaler function
def MinMaxScaler(x):

  # Pre-compute the min and max of the variable
  min_x = np.min(x)
  max_x = np.max(x)

  # Calculate the newly scaled version of the variable
  u = (x - min_x) / (max_x - min_x)

  # Return the scaled version of the value
  return u

## SciKit-Learn
- Unless we are doing something tailored to a particular task, we generally don't want to code our own algorithms: It is time consuming, and existing implementations are typically more efficient or robust than what we would create (but there is a lot of value in coding your own algorithms and estimators)
- The most popular Python machine learning library is called **SciKit-Learn**
- You typically import it as
  - `from sklearn.<model class> import <model name>`
  - where `<model class>` is the set of related models and `<model name>` is the desired algorithm

- For k nearest neighbors we imported it as such, where neighbors is our `<model class>` and KNeighborsClassifier is our `<model name>`:
  - `from sklearn.neighbors import KNeighborsClassifier`

#### General Workflow for Sklearn with KNN Classification

1. Create an untrained model object with a fixed $k$:
  * For classification: `model = KNeighborsClassifier(n_neighbors=k)`
  * For regression: `model = KNeighborsRegressor(n_neighbors=k)`


2. Fit that object to the data, $(X,y)$:
  * `fitted_model = model.fit(X,y)`

3. Use the fitted object to make predictions for new cases $\hat{x}$ as either hard or soft classifications:
  * Hard Classification: `y_hat = fitted_model.predict(x_hat)`
  * Soft Classification: `y_hat = fitted_model.predict_proba(x_hat)`

## Workflow for fitting a of KNN Classifier Model

### 1. Loading in Our Data

* You can get the data from the GitHub repository for this class.

* For this notebook, we'll work with some data for classifying whether a patient tested positve for diabetes or not. The data come from female patients at least 21 years old of Pima Indian heritage. The available features are:

1. **Pregnancies**: Number of times pregnant
1. **Glucose**: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
1. **BloodPressure**: Diastolic blood pressure (mm Hg)
1. **SkinThickness**: Triceps skin fold thickness (mm)
1. **Insulin**: 2-Hour serum insulin (mu U/ml)
1. **BMI**: Body mass index (weight in kg/(height in m)^2)
1. **DiabetesPedigreeFunction**: Diabetes pedigree function. Indicates the function which scores likelihood of diabetes based on family history.
1. **Age**: Age (years)


In [ ]:
diabetes_df = pd.read_csv('./data/diabetes.csv', low_memory = False)
diabetes_df.head()

In [ ]:
# Get the shape of the data
diabetes_df.shape

In [ ]:
# Get the data types
diabetes_df.dtypes

In [ ]:
# Look at unique values of some variables
var = 'BloodPressure'
diabetes_df[var].unique()
diabetes_df[var].plot.hist(bins = 20)

# Note that Glucose, BloodPressure, SkinThickness, Insulin, and BMI cannot be 0
# so these are effectively nan values. We want to impute them somehow.

In [ ]:
# Impute each case where the values are 0 and cannot feasibly be
variables_to_impute = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Loop over the variables to impute
for var in variables_to_impute:
  diabetes_df[var] = diabetes_df[var].mask(
      diabetes_df[var] == 0, # Boolean for whether the variable equals 0 for observation
      diabetes_df[var].median() # Replacing with the median for the variable, robust to outliers
  )

In [ ]:
# Check the result after the imputation
var = 'BloodPressure'
diabetes_df[var].unique()
diabetes_df[var].plot.hist(bins = 20)

### 2. Select Our Outcome and Feature variables, Normalize the features

In [ ]:
# Select the outcome variable
y = diabetes_df['Outcome']

# Select the feature variables
var1 = 'Glucose'
var2 = 'Insulin'
features_of_interest = [var1, var2]

# Isloate our covariates/features
x = diabetes_df.loc[:, features_of_interest]

# Scale our variables using the min/max scaler
u = x.apply(MinMaxScaler)


In [ ]:
# Compare the scales of the original and re-scaled values
sns.histplot(x[var1], label = f'{var1}')
sns.histplot(x[var2], label = f'{var2}')
plt.xlabel(f'Values for {var1} and {var2}')
plt.title(f'Unscaled Distributions')
plt.legend()
plt.show()

sns.histplot(u[var1], label = f'{var1}')
sns.histplot(u[var2], label = f'{var2}')
plt.xlabel(f'Scaled Values for {var1} and {var2}')
plt.title(f'Scaled Distributions')
plt.legend()
plt.show()



In [ ]:
# Look at the scatter points of the two variables of interest to guess what the
# knn result will be

sns.scatterplot(x = u[var1], y = u[var2], hue = y)
plt.title(f'Comparison of Outcome: {var1} vs {var2}')
plt.show()

### 3. Fit a kNN Classifier and Predict

In [ ]:
# Set the number of neighbors, use an odd number to break ties
k = 5

# Create a fitted model instance
model = KNeighborsClassifier(n_neighbors = k)

# Fit the model using our scaled data and our outcome variable
model = model.fit(u, y)

# Make predictions using the fit model
y_hat = model.predict(u)

#### Visualize our predictions

In [ ]:
# Visualize what we predicted
# Here, the color is the true label
# The marker style (o vs x) is the prediction
sns.scatterplot(
    x = u[var1],
    y = u[var2],
    hue = y, # Setting the marker color based on the truth
    style = y_hat # Setting the marker style based on our prediction
)

plt.title('Comparison of True Label and our Prediction')
plt.show()


In [ ]:
# Visualize our predictions in the entire space of possible observations

nx = 100 # Number of points on grid for x variable
ny = 100 # Number of points on grid for y variable
total = nx*ny # Total number of points to plot

grid_x = np.linspace(0, 1, nx) # Create a grid of x values
grid_y = np.linspace(0, 1, ny) # Create a grid of y values

xs, ys = np.meshgrid(grid_x, grid_y) # Explode grids to all possible pairs
X = xs.reshape(total) # Turns pairs into vectors
Y = ys.reshape(total) # Turns pairs into vectors

# Create a dataframe of points to plot
x_hat = pd.DataFrame(
    {
        var1: X,
        var2: Y
    }
)

# Fit the model to the points
y_hat = model.predict(x_hat)

# Add new variable to the dataframe
x_hat['y_hat'] = y_hat

# Plot the values
this_plot = sns.scatterplot(
    data=x_hat,
    x=var1,
    y=var2,
    hue='y_hat',
    linewidth=0
)

plt.xlabel(var1)
plt.ylabel(var2)
plt.title('Prediction Grid Map')

# Move legend off the plot canvas
sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1))

#### Look at whether our model worked: Confusion Matrices and Accuracy

* Once we create a model, we want to gain an understanding about whether the model worked.

* When we are working with a classification task (ex. whether a patient has diabetes or not), a basic metric to look at is the confusion matrix. We cross-tabulate the true labels and the predicted labels, hoping to see that they align on the diagonal.

In [ ]:
# Create a cross tab between our true values and the predicted values
# We can do normalize to see what percentage of observations fell within
# each section
cross = pd.crosstab(y, y_hat, normalize = True)
cross

In [ ]:
# We can visualize this as a heatmap, where color is based on the percentage
# of observations in that category
sns.heatmap(cross.values, annot = True, cmap = 'Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

* Sometimes we want to simplify this confusion matrix into a single, summary number. The simplest and most obvious method to do this is to look at what proportion of the cases we predicted properly. This is called **accuracy.**

* We can calculate the accuracy by summing the numbers on the diagonal of the confusion matrix. If they're raw counts, we need to divide by the total number of observations. If they're already proportions, the sum is sufficient.

In [ ]:
# We can combine this into a single value, the Accuracy
# We sum the diagonal of the confusion matrix
accuracy = np.sum(np.diag(cross))
print('Accuracy:', accuracy)

## Selecting $k$

### Hyperparameters

- For k nearest neighbors, k is a value that we pick and is not learned by the model. This is called a **hyper-parameter**. We don't have an obvious way to pick what the value for k should be just by looking at the data or model itself.
- We need to do tests to identify what the best value of k should be for our model. It will have a big effect on the model outcomes.
- Usually the most complex models will do the best on the data we have (training data), but they will do a poor job when generalizing to unseen data. We need to strike a delicate balance. In general, we want to find the most parsimonious (simple) model that we can that still does a good job on the test data.


Let's look at what happens to the accuracy when we modify the value for k:

In [ ]:
# Create a grid of odd numbered ks
k_grid = np.array([(2*k + 1)**2 for k in range(0, 11)])

# List to save the accuracies
accuracies = []

# Loop over the values of k
for k in k_grid:
  model = KNeighborsClassifier(n_neighbors = k) # Define model instance
  model = model.fit(u, y) # Fit the model
  y_hat = model.predict(u) # Make predictions
  correct = (y == y_hat).astype(int) # Find which predictions are correct

  # Plot the values
  sns.scatterplot(
      x = diabetes_df[var1],
      y = diabetes_df[var2],
      hue = correct,
      style = correct
  )
  plt.title(f'Comparison of {var1} vs {var2}\nk = {k}')
  plt.show()

  # Compute the accuracy
  acc = model.score(u, y)

  # Save the accuracy
  accuracies.append(acc)

# Plot the accuracy over time
sns.lineplot(x = k_grid, y = accuracies)
plt.title('Accuracy vs k')
plt.xlabel('k')
plt.show()

#### Take-away from the above plots?
  - From this plot it looks like we should pick k = 1 because it gave us the highest accuracy on our training data set.
  - What does this mean in the algorithm? The prediction is based on the single nearest observation in our training data.
  - As we increase k, we take into account more nearby points and start to generalize.
  - Is it actually true that we should use k=1? How will the perform on unseen data?

### Overfitting vs Underfitting

* Overfitting and underfitting are common phenomenon when training a machine learning model that can negatively impact the outcome, making our models worse in practice.

* **Overfitting:** A case when our model is sensitive to a handful of data points found in our training data. In other words, the model learns patterns that are specific to the training data that do not generalize to unseen data. When using the the k nearest neighbors algorithm, this occurs with a small value of k. Ex. $k=1$.

* **Underfitting:** A case when the model averages over many observations and gives answers close to population proportions. When using the k nearest neighbors algorithm, this occurs with a large value of k. Ex. $k = 400$

### Train-Test Splits

- In machine learning, we are interested in **prediction**. We want to have the best prediction for the outcome of new cases after training our model. We don't care as much about how our model performs on data we've already seen, we care more about how our model does on unseen data.

- Our plot that we created above is more concerned with how the model performs on data it has already seen. **How can we think about how the model will do on data it has not seen?**

- This is where **train-test** splits are handy. We split our data into train and test sets.
  - **Train Data:** This is the data that we actually train our model on. This is typically 80% of the data we have on hand, but the exact proportion can vary.
  - **Test data**: The data that we test our model on. We calculate the performance metrics (confusion matrices and accruacy) on the test data, simulating how the model would perform on unseen data. This is typically the remaining 20% of the data, but again this proportion can vary.

- To perform the train-test split, we will use the `train_test_spilt` function from sklearn. The documentation for the function can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
  

In [ ]:
# Sklearn has a function for creating the train test split for us
from sklearn.model_selection import train_test_split

In [ ]:
# Gathering the train test split, the order of how you catch these outputs matters
u_train, u_test, y_train, y_test = train_test_split(
    u, # our scaled covariates
    y, # our accompanying outcome values
    test_size = 0.2, # the proportion of the data
    random_state = 24 # the seed for the split, makes the results reproducible
)

In [ ]:
# Create the model instance and fit using the TRAIN data
model = KNeighborsClassifier(n_neighbors = 5)
model = model.fit(u_train, y_train)

# Get the prediction on the TEST set
y_hat = model.predict(u_test)

In [ ]:
# Look at the confusion matrix
cross = pd.crosstab(y_test, y_hat, normalize = True)

# We can visualize this as a heatmap, where color is based on the percentage
# of observations in that category
sns.heatmap(cross.values, annot = True, cmap = 'Blues')
plt.title('Confusion Matrix for Test Set')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# Calculate the accuracy for the TEST set
acc = model.score(u_test, y_test)
print('Accraucy on Test Set for k=5:', acc)

**Was our accuracy better or worse than on the test set?**

### Selecting $k$ using our Test Data

In [ ]:
# Select out some values of k to look at, keeping them odd
k_grid = np.array([(2*k + 3) for k in range(0, 100)]) # Start with a value of 3, 1 is not recommended

# Lists to save the values for the
# train and test accuracies
train_accuracies = []
test_accuracies = []

# Loop over the values of k
for k in k_grid:
  # Fitting the model on the TRAIN data
  model = KNeighborsClassifier(n_neighbors = k) # Define model instance
  model = model.fit(u_train, y_train)

  # Predict on the TRAIN data (helps us see how it's doing on the train data)
  y_hat_train = model.predict(u_train)
  train_acc = model.score(u_train, y_train)

  # Predict on the TEST data (what we're most interested)
  y_hat_test = model.predict(u_test)
  test_acc = model.score(u_test, y_test)

  # Append the results so that we can compare them latter
  train_accuracies.append(train_acc)
  test_accuracies.append(test_acc)

  print(f'For k={k}, train accuracy: {train_acc}; test accuracy: {test_acc}')

# Plot the accuracy over time
sns.lineplot(x = k_grid, y = train_accuracies, label = 'Train Accuracy')
sns.lineplot(x = k_grid, y = test_accuracies, label = 'Test Accuracy')
plt.title('Accuracy vs k for Train and Testing Data')
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.show()

### Finding the Optimal k from the above plot

In [ ]:
# We want to select the k that gives us the largest test accuracy

# Turn the test accuracies into a numpy array
test_accuracies = np.array(test_accuracies)

# Find where it is maximized
best_k_idx = np.argmax(test_accuracies)
best_k = k_grid[best_k_idx]

print(f'The best k for the test data set is {best_k}')
